【内容】

・データ読み込み

・目的変数と説明変数のテーブルデータに変換

・初期ハイパーパラメータ初期設定

・グリッドリサーチでハイパーパラメータ設定

・アンサンブル学習(https://potesara-tips.com/ensemble-voting/#toc10)


データ読み込み

In [32]:
from sklearn.datasets import load_breast_cancer#サンプルデータセットのライブラリ
import pandas as pd#テーブルデータや時系列データを操作するためのデータ構造と演算のライブラリ

data = load_breast_cancer()#データ読み込み
data#dataを表示

目的変数と説明変数のテーブルデータに変換


In [33]:
train_x = pd.DataFrame(data=data.data,columns=data.feature_names)#説明変数のテーブルデータに変換。データ:data,カラム:columns
train_x.head()#train_xのデータフレームの最初の5行表示

In [34]:
train_y = pd.DataFrame(data=data.target,columns=['class'])#目的変数のテーブルデータに変換。データ:data,カラム:columns
train_y.head()#train_yのデータフレームの最初の5行表示

In [4]:
from sklearn.metrics import log_loss, accuracy_score#混同行列を計算するライブラリ
from sklearn.model_selection import KFold#ホールドアウト、交差検証(cross validation)するライブラリ
import lightgbm as lgb#LightGBMのライブラリ
import numpy as np#数値計算するライブラリ

# 各foldのスコアを保存する空のリスト
scores_accuracy = []#[]:正解率用のリスト
scores_logloss = []#[]:評価関数LogLoss用のリスト

# クロスバリデーション(k-fold cross validation)を行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=3, shuffle=True, random_state=42)#n_splits:分割,shuffle:シャッフルするかどうか,random_state:何度実行しても同じランダムな分割になるように固定する値。指定しないと毎回分割データが変わり結果が異なってしまう

確認用【for tr_idx, va_idx in kf.split(train_x):】

In [35]:
cnt = 1#カウント用の数字
for tr_idx, va_idx in kf.split(train_x):#クロスバリデーションで分割。for：繰り返し(n_splitsの回数繰り返す),in：どんな条件で繰り返すか
    print('########{}回目:データ数 tr:{},va:{}########'.format(cnt,len(tr_idx),len(va_idx)))#print:文字列を出力、len([]):リストの長さ
    print('tr_idx:{}'.format(tr_idx))#'{}'.format(変数)
    print('va_idx:{}'.format(va_idx))
    cnt += 1#cntを1足す。

初期ハイパーパラメータ初期設定

In [ ]:
for tr_idx, va_idx in kf.split(train_x):#クロスバリデーションで分割。for：繰り返し(n_splitsの回数繰り返す),in：どんな条件で繰り返すか
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]#iloc[]:行番号、列番号を指定して要素を取り出す
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = lgb.LGBMClassifier() # LightGBMのモデルを定義。分類:LGBMClassifier(),回帰:LGBMRegressor()
    model.fit(tr_x, tr_y) # モデルの学習:fit(説明変数,目的変数)

    # テストデータの予測クラス (予測クラス(0 or 1)を返す)
    y_pred = model.predict(va_x)
    # テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
    y_pred_prob = model.predict_proba(va_x)    

    # モデル評価
    # acc : 正答率
    accuracy = accuracy_score(va_y,y_pred)#accuracy_score(正解値,予測値)
    print('accuracy :',accuracy)

    # 評価関数LogLoss 
    logloss =  log_loss(va_y,y_pred_prob) # 引数 : log_loss(正解クラス,[クラス0の予測確率,クラス1の予測確率])
    print('logloss :', logloss)

    # 各foldのスコアを保存する
    scores_logloss.append(logloss)#append:listに追加
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
print('scores_logloss:{}'.format(scores_logloss))
print('scores_accuracy:{}'.format(scores_accuracy))
logloss = np.mean(scores_logloss)#各foldの評価関数LogLossの平均値
accuracy = np.mean(scores_accuracy)#各foldの正解率の平均値
print("===予測モデルの評価指標(平均スコア)===")
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')#f'':変数をそのまま指定できる,{*.4f}:小数点以下の桁数(この場合4桁)

グリッドリサーチでハイパーパラメータ設定

In [ ]:
from sklearn.model_selection import GridSearchCV#グリッドサーチのライブラリ

model = lgb.LGBMClassifier() # LightGBMのモデルを定義
kf = KFold(n_splits=3, shuffle=True, random_state=42)#n_splits:分割,shuffle:シャッフルするかどうか,random_state:何度実行しても同じランダムな分割になるように固定する値。指定しないと毎

# パラメーターを設定する
param_grid = {"max_depth": [5,6,7,8,9],#木構造の深さを限定するための変数．データが少ないときに過学習を防ぐために設定する
              "learning_rate" : [0.05,0.06,0.07,0.08,0.09],#学習率
              'num_leaves': [5,6,7,8,9]#木にある分岐の個数
             }
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
grid = GridSearchCV(estimator = model,#モデル
                    param_grid = param_grid,#パラメータ
                    scoring = 'accuracy',#評価指標
                    cv = kf,#交差検定の回数
                    verbose=1,#verbose=1:一定の間隔でログ表示，verbose=2:テスト毎にログ表示，verbose=3:テスト毎にスコアも含めてログ表示
                    return_train_score = True,#False場合、トレーニングスコアを含まない
                    n_jobs = -1)#同時実行数(-1にするとコア数で同時実行)。#コア数とはコンピュータのCPUに内蔵された稼働するプロセッサコアの数。 コアの数だけ複数のプログラムを並列に動作させられる

grid.fit(c)#学習

print("===予測モデルの評価指標(平均スコア)===")
print("最も良いパラメータ:{}".format(grid.best_params_))
print(f'accuracy: {grid.best_score_:.4f}')#f'':変数をそのまま指定できる,{*.4f}:小数点以下の桁数(この場合4桁)

アンサンブル学習(https://potesara-tips.com/ensemble-voting/#toc10)

In [8]:
import xgboost as xgb#機械学習モデル：xgboost
import lightgbm as lgb#機械学習モデル：lightgbm
from catboost import CatBoost#機械学習モデル：catboost
from catboost import Pool

In [10]:
#関数とは
#def 関数名(引数1, 引数2, ...):
#    処理
#    return 戻り値

def Add(a, b):
    x = a + b
    return x

x = Add(4, 1)
print(x)

5


In [16]:
def xgb_train(X_train, y_train, X_eval, y_eval, X_test, y_test):#def:関数定義
    # 学習用データセット作成
    xgb_train = xgb.DMatrix(X_train, label=y_train)#X_train_cv:目的変数のデータ,y_train_cv:説明変数のデータ
    # 検証用データセット作成
    xgb_eval = xgb.DMatrix(X_eval, label=y_eval)#X_eval_cv:目的変数のデータ,y_eval_cv:説明変数のデータ
    # テスト用データセット作成
    xgb_test = xgb.DMatrix(X_test, label=y_test)#X_test:目的変数のデータ,y_test:説明変数のデータ
    
    # パラメータを設定
    xgb_params = {
        'objective': 'multi:softprob',  # 多値分類問題
        #softprob:予測するそれぞれのクラスの確率値を出力[0.7,0.2,0.1]
        #softmax:予測が最大確率となっている1クラスを出力[0.7,0.2,0.1]→0    
        'num_class': 3,                 # 目的変数のクラス数
        'learning_rate': 0.1,           # 学習率
        'eval_metric': 'mlogloss'       # 学習用の指標 (Multiclass logloss)
    }

    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメータ
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=200,               # 学習の回数
                    early_stopping_rounds=10,          # アーリーストッピング
                    evals=evals,                       # 学習経過で表示する名称
                    evals_result=evaluation_results,   # 上記で設定した検証用データ
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    )
    
    # テストデータで予測
    y_pred = bst.predict(xgb_test, ntree_limit=bst.best_ntree_limit)#各クラスの予想確率#一番良いモデルで予測する
    y_pred_max = np.argmax(y_pred, axis=1)#最大値のインデックス（位置）を取得

    accuracy = accuracy_score(y_test.values.tolist(), y_pred_max)#クラス分類結果を評価
    print('XGBoost Accuracy:', accuracy)
    
    return(bst, y_pred_max, accuracy)#return:出力、bst:モデル、y_pred_max:予測値,acuracy:正解率

In [17]:
def lgbm_train(X_train, y_train, X_eval, y_eval, X_test, y_test):
    # データを格納する
    # 学習用
    lgb_train = lgb.Dataset(X_train, y_train,free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval, y_eval, reference=lgb_train,free_raw_data=False)
    # パラメータを設定
    params = {'task': 'train',                # レーニング ⇔　予測predict
              'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'multiclass',      # 目的関数：多値分類、マルチクラス分類
              'metric': 'multi_logloss',      # 検証用データセットで、分類モデルの性能を測る指標
              'num_class': 3,                 # 目的変数のクラス数
              'learning_rate': 0.1,           # 学習率（初期値0.1）
              'num_leaves': 23,               # 決定木の複雑度を調整（初期値31）
              'min_data_in_leaf': 1,          # データの最小数（初期値20）
              'verbose': -1,
             }

    # 学習
    evaluation_results = {}                                # 学習の経過を保存する箱
    model = lgb.train(params,                              # 上記で設定したパラメータ
                      lgb_train,                           # 使用するデータセット
                      num_boost_round=200,                 # 学習の回数
                      valid_names=['train', 'valid'],      # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],    # モデルの検証に使用するデータセット
                      evals_result=evaluation_results,     # 学習の経過を保存
                      early_stopping_rounds=10,            # アーリーストッピングの回数
                      verbose_eval=0)                      # 学習の経過を表示する刻み（非表示）

    # テストデータで予測
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_max = np.argmax(y_pred, axis=1)
    # Accuracy の計算
    accuracy = accuracy_score(y_test, y_pred_max)
    print('LightGBM Accuracy:', accuracy)
    
    return(model, y_pred_max, accuracy)#return:出力、model:モデル、y_pred_max:予測値,acuracy:正解率

In [18]:
def catboost_train(X_train, y_train, X_eval, y_eval, X_test, y_test):
    # データを格納する
    # 学習用
    CatBoost_train = Pool(X_train, label=y_train)
    # 検証用
    CatBoost_eval = Pool(X_eval, label=y_eval)
    
    # パラメータを設定
    params = {        
        'loss_function': 'MultiClass',    # 多値分類問題
        'num_boost_round': 1000,          # 学習の回数
        'early_stopping_rounds': 10       # アーリーストッピングの回数
    }
    
    # 学習
    catb = CatBoost(params)
    catb.fit(CatBoost_train, eval_set=[CatBoost_eval], verbose=False)

    # テストデータで予測
    y_pred = catb.predict(X_test, prediction_type='Probability')
    y_pred_max = np.argmax(y_pred, axis=1)

    # Accuracy の計算
    accuracy = accuracy_score(y_test, y_pred_max)
    #accuracy = sum(y_test == y_pred_max) / len(y_test)
    print('CatBoost Accuracy:', accuracy)
    
    return(catb, y_pred_max, accuracy)#return:出力、catb:モデル、y_pred_max:予測値,acuracy:正解率

In [ ]:
import xgboost as xgb#機械学習モデル：xgboost
import lightgbm as lgb#機械学習モデル：lightgbm
from catboost import CatBoost#機械学習モデル：catboost
from catboost import Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=train_y)

# Voting
# 各3つのモデルを保存するリスト
xgb_models = []
lgbm_models = []
catb_models = []
# 各3つのモデルの正答率を保存するリスト
xgb_accuracies = []
lgbm_accuracies = []
catb_accuracies = []
# 学習のカウンター
loop_counts = 1

# 各3つのモデルの予測を保存する配列の初期化（kfoldの3*3モデル）
first_preds = np.zeros((len(y_test), 3*3))

# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って3分割する）
K_fold = StratifiedKFold(n_splits=3, shuffle=True,  random_state= 1)

# KFoldクラスで分割した回数だけ実行（ここでは3回）
for train_no, eval_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_no, :]
    y_train_cv = y_train.iloc[train_no]
    X_eval_cv = X_train.iloc[eval_no, :]
    y_eval_cv = y_train.iloc[eval_no]

    # XGBoostの学習を実行
    bst, bst_pred, bst_accuracy = xgb_train(X_train_cv, y_train_cv,
                                            X_eval_cv, y_eval_cv, 
                                            X_test, y_test)
    # LIghtGBMの学習を実行
    model, model_pred, model_accuracy = lgbm_train(X_train_cv, y_train_cv,
                                                   X_eval_cv, y_eval_cv,
                                                   X_test, y_test)
    # CatBoostの学習を実行
    catb, catb_pred, catb_accuracy = catboost_train(X_train_cv, y_train_cv,
                                                    X_eval_cv, y_eval_cv,
                                                    X_test, y_test)

    # 学習が終わったモデルをリストに入れておく
    xgb_models.append(bst) 
    lgbm_models.append(model) 
    catb_models.append(catb) 

    # 学習が終わったモデルの正答率をリストに入れておく
    xgb_accuracies.append(bst_accuracy) 
    lgbm_accuracies.append(model_accuracy) 
    catb_accuracies.append(catb_accuracy) 

    # 学習が終わったモデルの予測をリストに入れておく
    first_preds[:, loop_counts-1] = bst_pred
    first_preds[:, loop_counts-1 + 3] = model_pred
    first_preds[:, loop_counts-1 + 6] = catb_pred

    # 実行回数のカウント
    loop_counts += 1  

In [ ]:
# 単独のモデルでの、テストデータの正答率
print('XGBoost Accuracy: ', np.array(xgb_accuracies).mean())#平均値算出
print('LightGBM Accuracy: ', np.array(lgbm_accuracies).mean())#平均値算出
print('CatBoost Accuracy: ', np.array(catb_accuracies).mean())#平均値算出

In [ ]:
# アンサンブルのモデルでの、テストデータの正答率
import itertools
# データ格納用のnumpy行列を作成
first_preds_max = pd.DataFrame(np.zeros((len(y_test), 3)))

# 予測したクラスのデータをpandas.DataFrameに入れる
df_first_preds = pd.DataFrame(first_preds)

# 各列（0,1,2）に、そのクラスを予測したモデルの数を入れる
first_preds_max.iloc[:, 0] = (df_first_preds == 0).sum(axis=1)
first_preds_max.iloc[:, 1] = (df_first_preds == 1).sum(axis=1)
first_preds_max.iloc[:, 2] = (df_first_preds == 2).sum(axis=1)

# 各行で、そのクラスを予測したモデルの数が最も多いクラスを得る
pred_max = np.argmax(np.array(first_preds_max), axis=1)

# Accuracy を計算する
#accuracy = sum(y_test.values.tolist() == pred_max) / len(y_test)
accuracy = accuracy_score(y_test, pred_max)
print('accuracy:', accuracy)
#df_accuracy = pd.DataFrame({'va_y': list(itertools.chain.from_iterable(y_test.values)),
#                            'y_pred_max': pred_max})
#print(pd.crosstab(df_accuracy['va_y'], df_accuracy['y_pred_max']))